# **Catégorisez automatiquement des questions**
Stack Overflow est un site célèbre de questions-réponses liées au développement informatique. Pour poser une question sur ce site, il faut entrer plusieurs tags afin de retrouver facilement la question par la suite. Pour les utilisateurs expérimentés, cela ne pose pas de problème, mais pour les nouveaux utilisateurs, il serait judicieux de suggérer quelques tags relatifs à la question posée.

Amateur de Stack Overflow, qui nous a souvent sauvé la mise, nous décidons d'aider la communauté en retour. Pour cela, nous allons développer un système de suggestion de tags pour le site. Celui-ci prendra la forme d’un algorithme de machine learning qui assignera automatiquement plusieurs tags pertinents à une question.

Stack Overflow propose un outil d’export de données, ["StackExchange Data Explorer"](https://data.stackexchange.com/stackoverflow/query/new), qui recense un grand nombre de données authentiques de la plateforme d’entraide. 

# **Sommaire**

*   **Partie 1:** <a href="#C1">Importation des données</a>
*   **Partie 2:** <a href="#C2">Modèle de prédiction des tags</a>
    * <a href="#C3">Nettoyage</a>
    * <a href="#C4">Représentation bag of words</a>
    * <a href="#C5">Méthodes de prédiction</a>
    * <a href="#C6">Prédiction finale des tags</a>

# <a name="C1">Partie 1: Importation des données</a>
On commence par importer les librairies nécessaires ainsi que les modèles et données pré-entraînés et sauvegardés.

In [ ]:
import pandas as pd
import numpy as np
import nltk

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from joblib import load

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

pd.set_option('display.max_columns', 100)

In [ ]:
classifiers = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/P5_BOURBON_Vicente/dict_classifiers.pkl')
vector_count_titles = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/P5_BOURBON_Vicente/Vector_count_titles.pkl')
vector_count_questions = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/P5_BOURBON_Vicente/Vector_count_questions.pkl')
LABELS_titles = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/P5_BOURBON_Vicente/labels_titles.pkl')
LABELS_questions = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/P5_BOURBON_Vicente/labels_questions.pkl')
stop_words = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/P5_BOURBON_Vicente/stop_words.pkl")

# <a name="C2">Partie 2: Modèle de prédiction des tags
Dans notre application finale, nous disposerons de deux entrées: le titre et le corps de la question. 

## <a name="C3">2.1: Nettoyage</a>
La première étape lorsqu'on récupère le titre et la question est de supprimer les stop words et les majuscules puis de tokeniser et lemmatiser le texte. 

In [ ]:
#Tokenization
def tokenizer_fct(sentence) :
  """Fonction qui prend en entrée une chaîne de caractères et renvoie la liste des tokens de cette chaîne"""

  sentence_clean = sentence.replace('-', ' ').replace('+', ' ').replace('/', ' ').replace('#', ' ')
  word_tokens = word_tokenize(sentence_clean)
  return word_tokens

#Retirer les majuscules
def lower_start_fct(list_words):
  """Fonction qui prend en entrée une liste de chaînes et renvoie la liste de ces chaînes en lettres minuscules"""

  lw = [w.lower() for w in list_words]
  return lw

def stop_words_filter_fct(list_words):
  """Fonction qui prend en entrée une liste de chaînes et renvoie la liste de ces chaînes nettoyée des stopwords """

  filtered_words = [w for w in list_words if not w in stop_words]
  filtered_w = [w for w in filtered_words if ((len(w) > 3) or (w=='c'))] #on conserve 'c' car il s'agit d'un nom de langage informatique
  return filtered_w

#Lemmatizer (base d'un mot)
def lemmatizer_fct(list_words):
  """Fonction qui prend en entrée une liste de chaînes et renvoie la liste après lemmatization"""

  lemmatizer = WordNetLemmatizer()
  lem_w = [lemmatizer.lemmatize(w) for w in list_words]
  return lem_w

#Fonction de préparation du texte pour le bag of words avec lemmatisation
def transform_bow_lem_fct(text) :
    word_tokens = tokenizer_fct(text)
    sw = stop_words_filter_fct(word_tokens)
    lw = lower_start_fct(sw)
    lem_w = lemmatizer_fct(lw) 
    transf_desc_text = ' '.join(lem_w)
    return transf_desc_text

## <a name="C4">2.2: Représentation bag of words</a>
Maintenant que les entrées sont nettoyées, il faut les représenter numériquement pour pouvoir utiliser les modèles de prédiction. Une représentaion classique bag of words est utilisée pour le direct matching et la régression logistique.

In [ ]:
def prepare_count_data(sentence, variable):
  """Fonction qui prend en entrée un texte ainsi que la variable indiquant s'il s'agit du titre ou de la question et le renvoie codé
  au format bag of words"""

  #Encodage avec le vecteur pré-entrainé adéquat
  if variable == 'Title':
    count_data = vector_count_titles.transform([sentence])
    tokens = vector_count_titles.get_feature_names_out()

  elif variable == 'Question':
    count_data = vector_count_questions.transform([sentence])
    tokens = vector_count_questions.get_feature_names_out()

  #Création du dataframe à retourner
  df = pd.DataFrame(count_data.toarray())
  df.columns = tokens

  return df

## <a name="C5">2.3: Méthodes de prédiction</a>
Les deux méthodes retenues qui seront utilisées en association sont le direct matching et la régression logistique.

In [ ]:
def direct_matching(series, variable):
  """Fonction qui prend en entrée du texte préalablement formaté ainsi que la variable indiquant s'il s'agit du titre ou de la question
  et qui renvoie la liste des tags apparaissant dans le texte du titre/question"""

  X_columns = set(series.columns)

  if variable == 'Title':
    y_columns = set(LABELS_titles)
  elif variable == 'Question':
    y_columns = set(LABELS_questions)
    
  #On initialise le tableau de sortie
  res = []
    
  #On parcourt les tokens et on met 1 dès qu'il y a match avec un tag
  common_columns = X_columns & y_columns
  for y_column in common_columns:
    for val in series[y_column]:
      if val > 0:
        res.append(y_column)
    
  return res

In [ ]:
def modele_classification(series):
  """Fonction qui prend en entrée le texte d'une question préalablement formaté en bag of words et renvoie la liste 
  des tags prédits"""

  x = series.to_numpy().reshape(1, -1)
  res = []
  for label in LABELS_questions:
    pred = classifiers[label].predict_proba(x)[0, 1]
    if pred >= 0.2:
      res.append(label)

  return res

In [ ]:
def prediction_direct_matching(text, variable):
  """Fonction qui prend en entrée du texte ainsi que la variable indiquant s'il s'agit du titre ou de la question et renvoie la liste de prédiction
  des tags"""
  
  text = transform_bow_lem_fct(text)
  series = prepare_count_data(text, variable)
  tags = direct_matching(series, variable)

  return tags

In [ ]:
def prediction_classification(text):
  """Fonction qui prend en entrée le texte d'une question et renvoie la liste de prédiction des tags"""

  text = transform_bow_lem_fct(text)
  series = prepare_count_data(text, 'Question')
  tags = modele_classification(series)

  return tags

## <a name="C6">2.4: Prédiction finale des tags</a>
On peut à présent écrire la fonction finale qui, en utilisant le titre et la question, retournera une proposition de tags.

In [ ]:
def prediction_tags(title, question):
  """Fonction qui prend en entrée le titre et la question et renvoie la liste des tags suggérés par les différents modèles
  de prédiction utilisés"""

  tags_titles = set(prediction_direct_matching(title, 'Title'))
  tags_questions = set(prediction_direct_matching(question, 'Question'))
  tags_questions_2 = set(prediction_classification(question))

  tags = list(set().union(tags_titles, tags_questions,tags_questions_2))
  tags = l = ' '.join(tags)

  return tags